# Diffusion starter

In [10]:
import numpy as np;
import matplotlib.pyplot as plt;
import scipy as sp;
from scipy.integrate import odeint
from scipy.optimize import dual_annealing
import plotly.graph_objects as go


## Determine gamma, y, v(0) from the attached data

$x'' = -\gamma x - yv$


In [7]:
df = pd.read_csv(r'C:\Users\emili\Downloads\unit5data-1.csv')
df.head()

,t,x
0,0.000,0.9897
1,0.101,1.4629
2,0.202,1.7087
3,0.303,1.8007
4,0.404,1.7923


#### Creating a mathematical model of a system of ODE's and comparing solution model to data

In [44]:
## Recommended approach:
## Create a mathematical model of a system of ODEs, v' = x'' and x' = v, and solve with odeint
## Compare your solution model to your data, and calculate goodness of fit
## Choose parameters that minimize your goodness of fit.
## If you create your goodness of fit calculation as a function which takes parameters
## you want to minimize as arguments, you could feed that to a minimization routine. I like
## dual_annealing for problems like this as it searches for a global min, and also allows you
## to input bounds which help to avoid the minimization getting too far out of the expected solution.

# Extract t and x values from the DataFrame
t_data = df['t'].values
x_data = df['x'].values

def model(y, t, gamma, y_value):
    x, v = y
    dydt = [v, -gamma * x - y_value * v]  # Define the system of ODEs
    return dydt

# Assuming you have initial conditions x0, v0, and values for gamma and y
x0 = x_data[0]
v0 = (x_data[1] - x_data[0]) / (t_data[1] - t_data[0])
gamma = 9
y_value = 0.5

# Initial conditions
y0 = [x0, v0]

# Solve the ODE system using odeint
solution = odeint(model, y0, t_data, args=(gamma, y_value))
x_solution = solution[:, 0]
v_solution = solution[:, 1]

# Plot the comparison between data and solution
fig = go.Figure()
fig.add_trace(go.Scatter(x=t_data, y=x_data, mode='lines', name='Data'))
fig.add_trace(go.Scatter(x=t_data, y=x_solution, mode='lines', name='Solution'))
fig.update_layout(
    title="Comparison of Data and Solution",
    xaxis_title="Time",
    yaxis_title="x",
    height = 500,
    width = 800
)
fig.show()


#### Calculating the goodness of fit using Sum of Squared Residuals (SSR)

In [46]:
# Calculate the sum of squared residuals (SSR)
ssr = np.sum((x_solution - x_data) ** 2)

# Print the SSR value
print("Sum of Squared Residuals (SSR):", ssr)

## print statement:
## Sum of Squared Residuals (SSR): 0.18585343971623475

## This SSR value indicates that the model fits the data pretty well. A perfect fit would be 0, but I doubt that we'll get that

Sum of Squared Residuals (SSR): 0.18585343971623475


#### Finding optimal parameters for best fit

In [51]:
## Choose parameters that minimize your goodness of fit
## In this case our parameters are gamma and y_value

## Create a function that takes these parameters as arguments, the function will serve as a minimization routine.
## Choosing dual_annealing to search for the global min.

def calculate_ssr(params):
    gamma = params[0]
    y_value = params[1]
    
    # Assuming you have initial conditions x0, v0
    x0 = x_data[0]
    v0 = (x_data[1] - x_data[0]) / (t_data[1] - t_data[0])
    
    # Initial conditions
    y0 = [x0, v0]
    
    # Solve the ODE system using odeint
    solution = odeint(model, y0, t_data, args=(gamma, y_value))
    x_solution = solution[:, 0]
    
    # Calculate the sum of squared residuals (SSR)
    ssr = np.sum((x_solution - x_data) ** 2)
    return ssr

# Define the bounds for gamma and y_value
bounds = [(0, 100), (0, 10)]

# Minimize the SSR using dual_annealing
result = dual_annealing(calculate_ssr, bounds=bounds)

# Get the optimal values of gamma and y_value
gamma_optimal = result.x[0]
y_value_optimal = result.x[1]

# Solve the ODE system with the optimal gamma and y_value
x0 = x_data[0]
v0 = (x_data[1] - x_data[0]) / (t_data[1] - t_data[0])
y0 = [x0, v0]
solution = odeint(model, y0, t_data, args=(gamma_optimal, y_value_optimal))
x_solution = solution[:, 0]

## Plot the comparison between data and optimal solution
fig = go.Figure()
fig.add_trace(go.Scatter(x=t_data, y=x_data, mode='lines', name='Data'))
fig.add_trace(go.Scatter(x=t_data, y=x_solution, mode='lines', name='Optimal Solution'))
fig.update_layout(
    title="Comparison of Data and Optimal Solution",
    xaxis_title="Time",
    yaxis_title="x",
    height = 500,
    width = 800
)
fig.show()

print(f"Optimal gamma: {gamma_optimal}")
print(f"Optimal y_value: {y_value_optimal}")

# Calculate the sum of squared residuals (SSR)
ssr = np.sum((x_solution - x_data) ** 2)

# Print the SSR value
print("Sum of Squared Residuals (SSR):", ssr)

Optimal gamma: 8.984328830122326
Optimal y_value: 0.48025199718743383
Sum of Squared Residuals (SSR): 0.1725562486954426


##### NOTE:

I want to make clear that I ended up using ChatGPT for most of the coding, I had a pretty good understanding of how to create, run and analyze these type of problems. All I had to do was explicitly ask ChatGPT what I wanted to create and how I wanted it create along with what the results I was expecting. ChatGPT basically falicitized the coding time. I want to be as honest as possible with the work I submit.